In [1]:
%load_ext autoreload
%autoreload

import io
import os
import sys
import django
import glob
import json
import re
import requests
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from tqdm.notebook import tqdm
from google.cloud import vision
from google.cloud import storage
from PIL import Image, ImageSequence
from gensim.models import Doc2Vec
from IPython.core.debugger import set_trace
import IPython
import collections
from googletrans import Translator
from google.cloud import translate_v2 as translate

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.db.models import F, Q, Sum, Count, Case, When, IntegerField, Value
from django.apps import apps
from django_currentuser.middleware import get_current_user
from datetime import datetime, timedelta
import time
import pandas as pd

In [2]:
#trans = m.Transaction.objects.values('sender__boo__user__email','receiver', 'when__date').annotate(point=Sum('amount'))

In [3]:
# trans = m.Transaction.objects.select_related('sender','receiver'); trans
trans = m.Transaction.objects.select_related(
    'sender','sender__boo','sender__boo__user','sender__raffle','sender__raffle__item','sender__support','sender__support__brand',
    'receiver','receiver__boo','receiver__boo__user','receiver__raffle','receiver__raffle__item','receiver__support','receiver__support__brand'
); trans

<QuerySet [<Transaction: quantlab | quantlab@kakao.com > KINTO TUMBLER 350ML - 39000 | 2021-08-07 20:38:17>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > STARBUCKS LAMP BK - 0 | 2021-08-07 20:37:53>, <Transaction: quantlab | quantlab@kakao.com > TAW&TOE ZEROVITY™ Flip Flop, SIZE M - 38000 | 2021-08-07 23:34:16>, <Transaction: quantlab | quantlab@kakao.com > KINTO TUMBLER 350ML - 39000 | 2021-08-07 20:38:17>, <Transaction: quantlab | quantlab@kakao.com > JOHNBER | 존버 스포츠 | 2021-08-10 18:02:20>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>,

In [63]:
# trans_dict = [{ 
#     'sender': str(_t.sender),
#     'receiver': str(_t.receiver),
#     'when': str(_t.when.date()),
#     'type': _t.get_type_display(),
#     'amount': _t.amount
# } for _t in trans[:10]]; trans_dict

In [4]:
trans_list = []
for _t in trans:
    tr = {}
    tr_type = _t.get_type_display()
    tr['user'] = str(_t.sender) if tr_type in ['래플','후원'] else str(_t.receiver)
    tr['when'] = str(_t.when.date())
    tr['type'] = tr_type
    tr['amount'] = -_t.amount if tr_type in ['래플','후원'] else _t.amount
    trans_list.append(tr)

In [82]:
pd.DataFrame(trans_list).groupby(['user','type','when']).sum().unstack().fillna(0).to_excel('transactions_amount.xlsx')

In [85]:
pd.DataFrame(trans_list).groupby(['user','type','when']).count().unstack().fillna(0).to_excel('transactions_count.xlsx')

In [30]:
counts = pd.DataFrame(trans_list).groupby(['user','type','when']).count().unstack().fillna(0)
counts.groupby('type').sum()[counts.columns[-11:-1]].drop(['첫방문환영'])

amount                                                         \
when    2021-09-13 2021-09-14 2021-09-15 2021-09-16 2021-09-17 2021-09-18   
type                                                                        
관심스타일입력        0.0        1.0        0.0        1.0        0.0        1.0   
관심아이템입력        0.0        0.0        0.0        1.0        0.0        1.0   
기본정보입력         3.0        0.0        2.0        3.0        2.0       10.0   
래플             0.0        0.0        3.0        2.0       12.0        7.0   
리서치참여         32.0        6.0       23.0       80.0       18.0       58.0   
매일밸런스게임      402.0      243.0      499.0      712.0     1000.0      639.0   
옷장넣기           0.0        0.0        0.0        2.0        3.0        4.0   
출첵게임          30.0       18.0       19.0       27.0       37.0       21.0   
후원             0.0        0.0        5.0        6.0        3.0        0.0   

                                                     
when    2021-09-19 2021-09-20 2021-09-21 2021-09-22  
type                                                 
관심스타일입력        1.0        0.0        0.0        0.0  
관심아이템입력        1.0        0.0        0.0        0.0  
기본정보입력         1.0        0.0        2.0        3.0  
래플            13.0        0.0        1.0        2.0  
리서치참여         30.0        8.0       33.0       19.0  
매일밸런스게임      751.0      819.0      775.0      886.0  
옷장넣기           6.0        1.0        1.0        3.0  
출첵게임          15.0       16.0       24.0       25.0  
후원             0.0        0.0        0.0        0.0

In [31]:
amounts = pd.DataFrame(trans_list).groupby(['user','type','when']).sum().unstack().fillna(0)
amounts.groupby('type').sum()[counts.columns[-11:-1]].drop(['첫방문환영'])

amount                                                         \
when    2021-09-13 2021-09-14 2021-09-15 2021-09-16 2021-09-17 2021-09-18   
type                                                                        
관심스타일입력        0.0       50.0        0.0       50.0        0.0       50.0   
관심아이템입력        0.0        0.0        0.0       50.0        0.0       50.0   
기본정보입력       300.0        0.0      200.0      300.0      200.0      400.0   
래플             0.0        0.0     -288.0     -143.0    -1050.0     -641.0   
리서치참여       3640.0      180.0     2020.0     8010.0     2140.0     6270.0   
매일밸런스게임      400.0      243.0      475.0      700.0      900.0      635.0   
옷장넣기           0.0        0.0        0.0        2.0        3.0        4.0   
출첵게임         240.0      140.0      130.0      230.0      230.0      190.0   
후원             0.0        0.0     -500.0     -600.0     -300.0        0.0   

                                                     
when    2021-09-19 2021-09-20 2021-09-21 2021-09-22  
type                                                 
관심스타일입력       50.0        0.0        0.0        0.0  
관심아이템입력       50.0        0.0        0.0        0.0  
기본정보입력       100.0        0.0      200.0      300.0  
래플         -1398.0        0.0      -77.0     -222.0  
리서치참여       2770.0      510.0     3670.0     1380.0  
매일밸런스게임      685.0      801.0      735.0      846.0  
옷장넣기           6.0        1.0        1.0        3.0  
출첵게임         110.0      140.0      180.0      210.0  
후원             0.0        0.0        0.0        0.0

In [37]:
qrs = m.User.objects.filter(date_joined__gte='2021-09-13').values('date_joined__date').annotate(count=Count('date_joined__date'))

In [45]:
pd.DataFrame(qrs).set_index('date_joined__date').sort_index(ascending=True)

,count
date_joined__date,
2021-09-13,9
2021-09-14,3
2021-09-15,7
2021-09-16,14
2021-09-17,13
2021-09-18,10
2021-09-19,7
2021-09-20,6
2021-09-21,12


In [28]:
df2.pivot(index='받기', columns='날짜', values='포인트')

날짜,2021-08-15,2021-08-16,2021-08-17,2021-08-18,2021-08-19,2021-08-20,2021-08-21,2021-08-22,2021-08-23,2021-08-24,...,2021-09-04,2021-09-05,2021-09-06,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13
받기,,,,,,,,,,,,,,,,,,,,,
212me2u2@hanmail.net,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30632phb@naver.com,NaN,NaN,NaN,NaN,340.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365ha@jr.naver.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN
9410252@naver.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
angelica.yum@gmail.com,NaN,NaN,NaN,NaN,1381.0,126.0,37.0,412.0,24.0,NaN,...,NaN,NaN,24.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yellowin@skmns.co.kr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
yoppei1001@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN
young_ee81@hotmail.com,NaN,NaN,NaN,NaN,NaN,340.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
points = m.Transaction.objects.values('sender','receiver', 'when__date').annotate(
#     point=Sum(Case(
#         When(), 
#         default=Value(0), 
#         output_field=IntegerField()
#     ))
    points=Sum('amount'),
#     sender=Value('sender')
)
points

<QuerySet [{'sender': 14, 'receiver': 46, 'when__date': datetime.date(2021, 8, 24), 'points': 91}, {'sender': 18, 'receiver': 11, 'when__date': datetime.date(2021, 8, 20), 'points': 100}, {'sender': 18, 'receiver': 4, 'when__date': datetime.date(2021, 8, 20), 'points': 77}, {'sender': 18, 'receiver': 2, 'when__date': datetime.date(2021, 8, 24), 'points': 66}, {'sender': 146, 'receiver': 13, 'when__date': datetime.date(2021, 9, 7), 'points': 111}, {'sender': 143, 'receiver': 13, 'when__date': datetime.date(2021, 9, 2), 'points': 111}, {'sender': 14, 'receiver': 139, 'when__date': datetime.date(2021, 9, 6), 'points': 430}, {'sender': 14, 'receiver': 108, 'when__date': datetime.date(2021, 8, 25), 'points': 223}, {'sender': 14, 'receiver': 18, 'when__date': datetime.date(2021, 9, 2), 'points': 10}, {'sender': 18, 'receiver': 3, 'when__date': datetime.date(2021, 8, 20), 'points': 111}, {'sender': 14, 'receiver': 23, 'when__date': datetime.date(2021, 9, 12), 'points': 10}, {'sender': 14, 're

In [38]:
from gensim.models import Doc2Vec

In [40]:
model = Doc2Vec.load('doc2vec.model')

In [44]:
model.wv.vocab

{'스투시': <gensim.models.keyedvectors.Vocab at 0x13931b970>,
 'stussy': <gensim.models.keyedvectors.Vocab at 0x13931bfd0>,
 '032c': <gensim.models.keyedvectors.Vocab at 0x13abad790>,
 '협업': <gensim.models.keyedvectors.Vocab at 0x13abad850>,
 '제품이': <gensim.models.keyedvectors.Vocab at 0x13abad5b0>,
 '공개되': <gensim.models.keyedvectors.Vocab at 0x13abad910>,
 '라이너': <gensim.models.keyedvectors.Vocab at 0x13abad970>,
 '마리아': <gensim.models.keyedvectors.Vocab at 0x13abad9d0>,
 'maria': <gensim.models.keyedvectors.Vocab at 0x13abada60>,
 'ilk': <gensim.models.keyedvectors.Vocab at 0x13abada90>,
 '스트릿패션': <gensim.models.keyedvectors.Vocab at 0x13abadac0>,
 '스트릿': <gensim.models.keyedvectors.Vocab at 0x13abadb80>,
 '패션': <gensim.models.keyedvectors.Vocab at 0x13abadbb0>,
 '스트릿브랜드': <gensim.models.keyedvectors.Vocab at 0x13abadc10>,
 '브랜드': <gensim.models.keyedvectors.Vocab at 0x13abadc70>,
 '한국': <gensim.models.keyedvectors.Vocab at 0x13abadcd0>,
 '최초': <gensim.models.keyedvectors.Vocab at 0x13